# Tutorial 1: Basic Pipeline Setup

Welcome to NeurOS! This tutorial will guide you through creating your first real-time neural decoding pipeline.

## What You'll Learn

1. Installing and importing NeurOS
2. Loading neural data
3. Creating a simple classification pipeline
4. Training a model
5. Making predictions
6. Evaluating results

## Prerequisites

- Python 3.9+
- Basic understanding of neural data
- Familiarity with NumPy and scikit-learn (helpful but not required)

## 1. Installation and Setup

First, let's make sure NeurOS is installed:

In [ ]:
# Install NeurOS (if not already installed)
# !pip install neuros

# Or install from local source (for development)
# !pip install -e .

## 2. Import Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns

# NeurOS imports
from neuros.pipeline import Pipeline
from neuros.drivers.mock_driver import MockDriver
from neuros.models.simple_classifier import SimpleClassifier
from neuros.datasets.allen_loader import load_simulated_allen_data

# Set random seed for reproducibility
np.random.seed(42)

print("✓ Imports successful!")

## 3. Load Simulated Neural Data

We'll start with simulated EEG data representing a 2-class motor imagery task:
- **Class 0**: Imagining left hand movement
- **Class 1**: Imagining right hand movement

In [ ]:
# Load simulated data
data = load_simulated_allen_data(
    n_samples=200,
    n_neurons=32,
    n_classes=2,
    noise_level=0.3
)

# Extract features and labels
X = data['features']  # Shape: (n_samples, n_features)
y = data['labels']    # Shape: (n_samples,)

print(f"Dataset shape: {X.shape}")
print(f"Number of classes: {len(np.unique(y))}")
print(f"Class distribution: {np.bincount(y)}")

## 4. Visualize the Data

Let's visualize some sample neural activity:

In [ ]:
# Plot first 5 samples from each class
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
fig.suptitle('Sample Neural Activity Patterns', fontsize=16)

for class_idx in range(2):
    class_samples = X[y == class_idx][:5]
    for sample_idx in range(5):
        ax = axes[class_idx, sample_idx]
        
        # Reshape to 2D for visualization (assuming time series)
        sample = class_samples[sample_idx].reshape(32, -1)
        
        ax.imshow(sample, aspect='auto', cmap='RdBu_r')
        ax.set_title(f'Class {class_idx}, Sample {sample_idx+1}')
        ax.set_xlabel('Time')
        ax.set_ylabel('Channel')
        
plt.tight_layout()
plt.show()

## 5. Split Data into Train and Test Sets

In [ ]:
from sklearn.model_selection import train_test_split

# 70% training, 30% testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

## 6. Create a NeurOS Pipeline

A Pipeline in NeurOS consists of:
- **Driver**: Handles data acquisition (real or simulated)
- **Model**: Performs classification/regression
- **Optional**: Preprocessing, feature extraction, etc.

In [ ]:
# Create driver (simulates real-time data acquisition)
driver = MockDriver(
    channels=32,
    sampling_rate=250.0,  # 250 Hz
    latency_ms=10.0
)

# Create classifier model
model = SimpleClassifier(
    model_type='svm',  # Support Vector Machine
    n_classes=2
)

# Create pipeline
pipeline = Pipeline(
    driver=driver,
    model=model
)

print("✓ Pipeline created successfully!")
print(f"Driver: {driver.__class__.__name__}")
print(f"Model: {model.__class__.__name__}")

## 7. Train the Model

Now let's train our classifier on the training data:

In [ ]:
# Train the pipeline
print("Training model...")
pipeline.train(X_train, y_train)
print("✓ Training complete!")

## 8. Make Predictions

Test the trained model on unseen data:

In [ ]:
# Make predictions on test set
y_pred = pipeline.predict(X_test)

print(f"Predictions shape: {y_pred.shape}")
print(f"First 10 predictions: {y_pred[:10]}")
print(f"First 10 true labels: {y_test[:10]}")

## 9. Evaluate Performance

Let's assess how well our model performs:

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.2%}")

# Detailed classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Left Hand', 'Right Hand']))

## 10. Confusion Matrix

Visualize which classes are being confused:

In [ ]:
# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Left Hand', 'Right Hand'],
            yticklabels=['Left Hand', 'Right Hand'])
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

## 11. Real-Time Simulation

Let's simulate real-time decoding for a few seconds:

In [ ]:
import asyncio

# Run pipeline in real-time mode for 3 seconds
print("Running real-time simulation...")
metrics = await pipeline.run(duration=3.0)

print("\nReal-Time Metrics:")
print(f"  Average latency: {metrics.get('latency_ms', 0):.2f} ms")
print(f"  Throughput: {metrics.get('throughput', 0):.1f} samples/sec")
print(f"  Predictions made: {metrics.get('n_predictions', 0)}")

## 12. Model Comparison (Bonus)

Let's compare different classifier types:

In [ ]:
from neuros.models.simple_classifier import SimpleClassifier

# Test different models
model_types = ['svm', 'random_forest', 'mlp']
results = {}

for model_type in model_types:
    print(f"\nTesting {model_type.upper()}...")
    
    # Create and train model
    model = SimpleClassifier(model_type=model_type, n_classes=2)
    pipeline = Pipeline(driver=driver, model=model)
    pipeline.train(X_train, y_train)
    
    # Evaluate
    y_pred = pipeline.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    results[model_type] = acc
    print(f"  Accuracy: {acc:.2%}")

# Plot comparison
plt.figure(figsize=(10, 6))
plt.bar(results.keys(), results.values())
plt.ylim([0, 1])
plt.ylabel('Accuracy')
plt.title('Model Comparison')
plt.axhline(y=0.5, color='r', linestyle='--', label='Chance level')
plt.legend()
plt.show()

print(f"\nBest model: {max(results, key=results.get)} ({max(results.values()):.2%})")

## 13. Save Your Model

Save the trained model for later use:

In [ ]:
import pickle
from pathlib import Path

# Create models directory
models_dir = Path('models')
models_dir.mkdir(exist_ok=True)

# Save model
model_path = models_dir / 'motor_imagery_classifier.pkl'
with open(model_path, 'wb') as f:
    pickle.dump(pipeline.model, f)

print(f"✓ Model saved to {model_path}")

## 14. Load and Use Saved Model

Load the model and make predictions:

In [ ]:
# Load saved model
with open(model_path, 'rb') as f:
    loaded_model = pickle.load(f)

# Create new pipeline with loaded model
new_pipeline = Pipeline(driver=driver, model=loaded_model)

# Make predictions
y_pred_loaded = new_pipeline.predict(X_test)
acc_loaded = accuracy_score(y_test, y_pred_loaded)

print(f"Loaded model accuracy: {acc_loaded:.2%}")
print("✓ Model successfully loaded and tested!")

## Summary

Congratulations! You've completed Tutorial 1. Here's what you learned:

✅ How to install and import NeurOS  
✅ Loading and visualizing neural data  
✅ Creating a Pipeline with Driver and Model  
✅ Training a classifier  
✅ Making predictions and evaluating performance  
✅ Running real-time simulations  
✅ Comparing different models  
✅ Saving and loading trained models  

## Next Steps

- **Tutorial 2**: Foundation Models Showcase - Learn about advanced pretrained models
- **Tutorial 3**: Multi-Modal Processing - Work with multiple data streams
- **Tutorial 4**: Custom Models - Build your own neural decoders

## Exercises

Try these challenges to deepen your understanding:

1. **Change the number of classes**: Modify the data loader to use 3 or 4 classes
2. **Tune hyperparameters**: Experiment with different SVM kernels or Random Forest parameters
3. **Add preprocessing**: Implement bandpass filtering or normalization
4. **Cross-validation**: Use k-fold cross-validation to get more robust accuracy estimates
5. **Feature importance**: Visualize which features are most important for classification

Happy coding! 🧠🚀